In [14]:


connection = sqlite3.connect("v30_koondkorpus_sentences_verb_pattern_obl_20240327-194533.db")  
cur = connection.cursor()





NameError: name 'sqlite3' is not defined

In [ ]:
#!pip install sqlalchemy
#!pip install mlxtend

In [199]:
from sqlalchemy import create_engine, text, MetaData, Table, select, and_
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

from mlxtend.frequent_patterns import apriori



class V30:
    _e = None
    _conn = None
    _matadata = None
    _te = TransactionEncoder()
    
    _t = {}
    
    def __init__(self, file_path):
        # relative path
        self._engine = create_engine(f'sqlite:///{file_path}')
        self._metadata = MetaData()
        self._conn = self._engine.connect()
        self.__init_tables()
        
        
    def __init_tables(self):
        q = text("SELECT name FROM sqlite_master WHERE type='table'")
        res = self._conn.execute(q).all()
       
        
        tables = [str(r[0]) for r in res]
        for t in tables:
            self._t[t] = Table(t, self._metadata, autoload_with=self._engine)
        
   
    def execute_text(self, q):
        return self._conn.execute(text(q))
    
    
    def execute(self, stmt):
        return self._conn.execute(stmt)
    
    
    
    def get_transactions(self, verb, verb_compound, columns=[], skip_deprels=[], include_deprels=[]):
        """
        Fetches transactions from the database and returns them as an array of dictionaries, each representing a transaction.

        Parameters:
        - verb (str): The main verb to filter transactions by.
        - verb_compound (str): Additional compound verb information for filtering.
        - columns (list of str, optional): Specifies which columns to include in each returned dictionary. If empty, all columns are included.
        - skip_deprels (list of str, optional): Dependency relations to exclude from the results.
        - include_deprels (list of str, optional): Dependency relations to include in the results. 
            If both skip_deprels and include_deprels are provided, include_deprels takes precedence.

        Returns:
        - list of dicts: A list where each dictionary represents a transaction, 
        structured according to the specified 'columns', or all transaction data 
        if 'columns' is empty or not provided. Transactions are grouped by 'head_id'.
        """
        
        
        TransactionHead = self._t['transaction_head']
        Transaction = self._t['transaction']
        
        
        where_filters = [TransactionHead.c.verb == verb]
        where_filters.append(TransactionHead.c.verb_compound == verb_compound)
        
        if isinstance(skip_deprels, list) and len(skip_deprels):
            where_filters.append(Transaction.c.deprel.notin_(skip_deprels))
        
        if isinstance(include_deprels, list) and len(include_deprels):
            where_filters.append(Transaction.c.deprel.in_(include_deprels))  
    
        # maybe some transaction_head field should be also includes in results eg TransactionHead.deprel
        selections = [Transaction]
        
        stmt = select(*selections)\
            .join(TransactionHead, TransactionHead.c.id == Transaction.c.head_id)\
            .where(and_(*where_filters))\
            .order_by(Transaction.c.head_id, Transaction.c.loc)
            
        transactions = {}
        for res in self.execute(stmt).mappings():
            res = dict(res)
            # group by transaction_head
            if res["head_id"] not in transactions:
                transactions[res["head_id"]] = []
                
            r_dict = dict(res)

            if columns:
                for key in list(r_dict.keys()):
                    if key not in columns:
                        del r_dict[key]
                        
            transactions[res["head_id"]].append(r_dict)
            
        return list(transactions.values())
    
   
    def dict_to_apriory(self, transactions):
        """
        Converts transaction data into a format suitable for Apriori algorithm processing.

        Parameters:
        - transactions (list of list of dicts): The transaction data, where each transaction is a list of row dictionaries.

        Returns:
        - list of list of tuples: A dataset where each transaction is represented as a list of tuples, with each tuple containing the row values.
        """
        return  [ [ tuple(row.values()) for row in tr] for tr in transactions ]
        
    
    def apriory(self, dataset, min_support=0.5, use_colnames=True):
        """
        Applies the Apriori algorithm on the dataset to find frequent itemsets based on a minimum support threshold.

        Parameters:
        - dataset (list of list of items): The transaction dataset for itemset generation.
        - min_support (float, optional): The minimum support threshold for itemsets to be considered frequent. Default is 0.5.
        - use_colnames (bool, optional): Indicates whether to use column names for itemset generation. Default is True.

        Displays:
        - A DataFrame of frequent itemsets sorted by their support values in descending order.
        - The transformed dataset DataFrame used for Apriori algorithm.
        """
        
        print('min_support:', min_support)
        te_ary = self._te.fit(dataset).transform(dataset)
        df = pd.DataFrame(te_ary, columns=self._te.columns_)
        display(apriori(df, min_support=min_support, use_colnames=use_colnames).sort_values('support', ascending=False))
        display(df)
        
    



In [201]:
v30 = V30("v30_koondkorpus_sentences_verb_pattern_obl_20240327-194533.db")

/var/folders/hh/s3kkmqlx7sb3js2bqncf08s00000gn/T/ipykernel_15534/4227891811.py:32: SAWarning: Skipped unsupported reflection of expression-based index `verb_compound`
  self._t[t] = Table(t, self._metadata, autoload_with=self._engine)
/var/folders/hh/s3kkmqlx7sb3js2bqncf08s00000gn/T/ipykernel_15534/4227891811.py:32: SAWarning: Skipped unsupported reflection of expression-based index `verb`
  self._t[t] = Table(t, self._metadata, autoload_with=self._engine)
/var/folders/hh/s3kkmqlx7sb3js2bqncf08s00000gn/T/ipykernel_15534/4227891811.py:32: SAWarning: Skipped unsupported reflection of expression-based index `loc`
  self._t[t] = Table(t, self._metadata, autoload_with=self._engine)
/var/folders/hh/s3kkmqlx7sb3js2bqncf08s00000gn/T/ipykernel_15534/4227891811.py:32: SAWarning: Skipped unsupported reflection of expression-based index `sentence_id`
  self._t[t] = Table(t, self._metadata, autoload_with=self._engine)


In [ ]:
transactions = v30.get_transactions(verb='kirjutama', verb_compound='alla', columns=['deprel','feats'], skip_deprels=['punct', 'cc'])
display(transactions[:1])

dataset = v30.dict_to_apriory(transactions)
v30.apriory(dataset, min_support=0.05)